<a href="https://colab.research.google.com/github/samibahig/IFT6390/blob/main/MutlNBAArka2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
import re
#import NLTK 
#import spacy

In [ ]:
pathTrain = '/content/train.csv'
pathTest = '/content/test.csv'

In [ ]:
def remove_non_ascii(words):
    return ''.join([i if ord(i) < 128 else ' ' for i in words])

In [ ]:
def process(df, t):
    df[t] = df[t].apply(lambda x : x.lower())
    #train['Abstract'] = train['Abstract'].apply(lambda x : remove_punctuation(x))
    df[t] = df[t].apply(lambda x : x.strip())
    df[t] = df[t].apply(lambda x : re.sub('\n', ' ', x))
    df[t] = df[t].apply(lambda x : re.sub('\[[^]]*\]', '', x))
    df[t] = df[t].apply(lambda x : re.sub("<.*?>", " ", x))
    df[t] = df[t].apply(lambda x : remove_non_ascii(x))
    df[t] = df[t].str.replace('[^\w\s]','')
    return df

In [ ]:
TrainingData = pd.read_csv(pathTrain)
TestingData = pd.read_csv(pathTest)
#TrainingData.shape
TestingData.shape

(15000, 2)

In [ ]:
DataAbstract = TrainingData['Abstract']
DataCategory = TrainingData["Category"]
X_Test = TestingData["Abstract"]

In [ ]:
process(TrainingData, 'Abstract')
process(TestingData, 'Abstract')

,Id,Abstract
0,0,we describe ways to define and calculate l_1no...
1,1,the progenitor systems of typeia supernovae sn...
2,2,omegawhite is a widefield high cadence synopti...
3,3,given n geq 2 and 1pn we consider the critical...
4,4,the motivation of this work is to improve the ...
...,...,...
14995,14995,we investigate the ability of the space infrar...
14996,14996,we study theoretically the influence of the su...
14997,14997,we present multifrequency radio observations o...
14998,14998,it is commonly accepted that the combination o...


In [ ]:
count_vect = CountVectorizer()

In [ ]:
X_train_counts = count_vect.fit_transform(DataAbstract)
X_train_counts.shape

(7500, 44526)

In [ ]:
X_Test_counts = count_vect.transform(X_Test)

In [ ]:
count_vect.vocabulary_.get(u'algorithm')

4917

In [ ]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [ ]:
tf_transformerTest = TfidfTransformer(use_idf=False).transform(X_Test_counts)

In [ ]:
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(7500, 44526)

In [ ]:
clf_Multinomial=MultinomialNB()
clf_Multinomial.fit(X_train_counts, DataCategory)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
print('The Train score for Multinomial is {0}'.format(clf_Multinomial.score(X_train_counts,DataCategory)))

The Train score for Multinomial is 0.9298666666666666


In [ ]:
pred = clf_Multinomial.predict(tf_transformerTest)

In [ ]:
pred.shape

(15000,)

In [ ]:
pd.DataFrame(pred).to_csv('MultinomialNB.csv')